# 📊 Phase 2 Results Visualization: 2x2 MNIST Grid ViT 🖼️➡️🔢🔢🔢🔢

**🎯 Goal:** Load the trained Vision Transformer model from Phase 2 and visualize its predictions on sample 2x2 grid images from the validation/test set. Analyze common errors.

## 1. Setup and Imports 🛠️🐍

In [13]:
import os
import sys
from pathlib import Path
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [30]:
# --- Add project root to sys.path for imports ---
# Assuming the notebook is run from the project root directory
project_root = Path(os.getcwd()).parent
src_path = project_root / 'src'
if str(project_root) not in sys.path:
    print(f"📂 Adding project root to sys.path: {project_root}")
    sys.path.insert(0, str(project_root))

# Import custom Vocabulary class and logger (Optional - if needed later, otherwise remove)
try:
    from utils import logger, load_config, get_device
    from src.mnist_transformer.dataset import get_mnist_dataset, MNISTGridDataset, get_mnist_dataloader, MNIST_MEAN, MNIST_STD 

    from src.mnist_transformer.model import VisionTransformer
except ImportError as e:
    print(f"⚠️ Error importing project logger: {e}")
    print("   Ensure you are running this notebook from the project root directory.")
    # Fallback logger if utils fails
    import logging
    logger = logging.getLogger("EDA_Notebook")
    logging.basicConfig(level=logging.INFO)
    logger.info("Using basic fallback logger.")

print("\n✅ Setup Complete.")


✅ Setup Complete.


## 2. Configuration & Model Loading ⚙️💾

Specify the path to the trained Phase 2 model checkpoint and load the configuration used during its training.

In [24]:
# --- Specify the Run/Model Path ---
# Use the exact name of the successful Phase 2 run directory
RUN_NAME = "Phase2_E20_LR0.0005_B128_ViT" # <-- UPDATE THIS LINE
MODEL_BASE_DIR = project_root / "models/mnist_vit"
MODEL_RUN_DIR = MODEL_BASE_DIR / RUN_NAME
CHECKPOINT_NAME = "model_final.pth"
CONFIG_PATH = project_root / "config.yaml" # Load the main config

model_checkpoint_path = MODEL_RUN_DIR / CHECKPOINT_NAME

# --- Load Config ---
logger.info(f"Loading configuration from: {CONFIG_PATH}")
config = load_config(config_path=CONFIG_PATH)
if config is None:
    raise FileNotFoundError("Config file not found or failed to load!")

# --- Device ---
device = get_device()
logger.info(f"Using device: {device}")

2025-04-28 17:26:09 | Backprop Bunch | INFO     | [3087244.py:12] | Loading configuration from: /Users/Oks_WORKSPACE/Desktop/DEV/W3_project/mlx-w3-mnist-transformer/config.yaml
2025-04-28 17:26:09 | Backprop Bunch | INFO     | [run_utils.py:22] | 🔍 Loading configuration from: /Users/Oks_WORKSPACE/Desktop/DEV/W3_project/mlx-w3-mnist-transformer/config.yaml
2025-04-28 17:26:09 | Backprop Bunch | INFO     | [run_utils.py:26] | ✅ Configuration loaded successfully.
2025-04-28 17:26:09 | Backprop Bunch | INFO     | [device_setup.py:38] | ✅ MPS device found and available (Built: True). Selecting MPS.
2025-04-28 17:26:09 | Backprop Bunch | INFO     | [device_setup.py:50] | ✨ Selected compute device: MPS
2025-04-28 17:26:09 | Backprop Bunch | INFO     | [3087244.py:19] | Using device: mps


In [25]:
# --- Instantiate Model ---
logger.info("Instantiating model architecture...")
model_cfg = config.get('model', {})
dataset_cfg = config.get('dataset', {})

p2_img_size = dataset_cfg.get('image_size_phase2', 56)
p2_patch_size = dataset_cfg.get('patch_size', 7) # Make sure this matches training
p2_num_outputs = dataset_cfg.get('num_outputs_phase2', 4)
p2_num_classes = dataset_cfg.get('num_classes', 10) # Make sure this matches

# --- 👇 Ensure these parameters match the TRAINED model ---
p2_embed_dim = model_cfg.get('embed_dim', 64)
p2_depth = model_cfg.get('depth', 6) # <-- CHANGE THIS LINE back to 4 (or whatever depth was used in the 'Phase2_E5...' training run)
p2_num_heads = model_cfg.get('num_heads', 4)
p2_mlp_ratio = model_cfg.get('mlp_ratio', 2.0)
# --- End Ensure ---

2025-04-28 17:26:13 | Backprop Bunch | INFO     | [2537617560.py:2] | Instantiating model architecture...


In [26]:
model = VisionTransformer(
    img_size=p2_img_size,
    patch_size=p2_patch_size,
    in_channels=dataset_cfg['in_channels'],
    num_classes=p2_num_classes,
    embed_dim=p2_embed_dim,
    depth=p2_depth,
    num_heads=p2_num_heads,
    mlp_ratio=p2_mlp_ratio,
    num_outputs=p2_num_outputs
)

2025-04-28 17:26:16 | Backprop Bunch | INFO     | [model.py:120] | 🧠 ViT initialized: img=56, patch=7, depth=4, heads=4, embed=64, outputs=4


In [27]:
# --- Load Trained Weights --- 
if model_checkpoint_path.exists():
    logger.info(f"💾 Loading model weights from: {model_checkpoint_path}")
    try:
        # Load state dict, making sure to map location correctly
        state_dict = torch.load(model_checkpoint_path, map_location=device)
        model.load_state_dict(state_dict)
        logger.info("✅ Model weights loaded successfully.")
        model.to(device) # Move model to device
        model.eval() # Set model to evaluation mode
    except Exception as e:
        logger.error(f"❌ Failed to load model weights: {e}", exc_info=True)
        model = None # Set model to None if loading fails
else:
    logger.error(f"❌ Model checkpoint not found at: {model_checkpoint_path}")
    model = None

2025-04-28 17:26:18 | Backprop Bunch | INFO     | [2954939614.py:3] | 💾 Loading model weights from: /Users/Oks_WORKSPACE/Desktop/DEV/W3_project/mlx-w3-mnist-transformer/models/mnist_vit/Phase2_E20_LR0.0005_B128_ViT/model_final.pth
2025-04-28 17:26:18 | Backprop Bunch | INFO     | [2954939614.py:8] | ✅ Model weights loaded successfully.


## 3. Load Phase 2 Validation/Test Data 🔢🔢🔢🔢

We need to generate some samples from the Phase 2 validation set (using the base MNIST test set) to see how the model performs.

In [28]:
# Load base MNIST test set
base_test_dataset = get_mnist_dataset(train=False, use_augmentation=False)

# Create Phase 2 Grid Dataset from the base test set
val_grid_dataset = None
if base_test_dataset:
    dataset_cfg = config.get('dataset', {})
    p2_img_size = dataset_cfg.get('image_size_phase2', 56)
    
    # Generate a reasonable number of validation samples
    num_val_samples = 500 
    val_grid_dataset = MNISTGridDataset(
        mnist_dataset=base_test_dataset, 
        length=num_val_samples, 
        grid_size=p2_img_size
    )
    logger.info(f"✅ Created Phase 2 validation dataset with {num_val_samples} samples.")
else:
    logger.error("❌ Cannot create Phase 2 validation set, base MNIST test set failed to load.")

# Create a DataLoader for easy batching
val_grid_loader = None
if val_grid_dataset:
    eval_batch_size = config.get('evaluation', {}).get('batch_size', 64) # Smaller batch for viz
    val_grid_loader = get_mnist_dataloader(
        val_grid_dataset, 
        batch_size=eval_batch_size, 
        shuffle=False, # No need to shuffle for validation
        num_workers=0 # Keep simple for notebook
    )
    logger.info(f"✅ Created Phase 2 validation DataLoader.")

2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:86] | 💾 Loading MNIST Test dataset...
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:87] |    Data directory: /Users/Oks_WORKSPACE/Desktop/DEV/W3_project/mlx-w3-mnist-transformer/data
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:89] |    Augmentation: Disabled (Test Set)
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:98] | ✅ MNIST Test dataset loaded successfully (10000 samples).
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:207] | 🧠 MNISTGridDataset initialized. Will generate 500 synthetic 56x56 images.
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [3158323562.py:17] | ✅ Created Phase 2 validation dataset with 500 samples.
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [dataset.py:127] | 📦 Creating DataLoader: batch_size=512, shuffle=False, num_workers=0, pin_memory=False
2025-04-28 17:26:21 | Backprop Bunch | INFO     | [3158323562.py:31] | ✅ Created Pha

## 4. Get Predictions & Visualize Results 👀🎯

Let's get predictions for a batch and visualize the input image, true labels, and predicted labels.

In [32]:
# --- Add imshow helper function here (copied from dataset.py test) ---
def imshow(img_tensor, title=''):
    """Helper function to display a single image tensor."""
    # Reverse the normalization for display purposes
    mean = torch.tensor(MNIST_MEAN)
    std = torch.tensor(MNIST_STD)
    img_tensor = img_tensor.cpu() # Move to CPU for numpy
    img_tensor = img_tensor * std[:, None, None] + mean[:, None, None]
    img_tensor = torch.clamp(img_tensor, 0, 1)
    npimg = img_tensor.numpy()
    plt.imshow(np.squeeze(npimg), cmap='gray') 
    plt.title(title)
    plt.axis('off')
# --- End imshow definition ---

In [33]:
if model is not None and val_grid_loader is not None:
    logger.info("Getting predictions for one batch...")
    
    images, true_labels = next(iter(val_grid_loader))
    images = images.to(device)
    # true_labels remain on CPU for comparison/plotting

    with torch.no_grad():
        outputs = model(images) # Get logits (B, 4, 10)
        
    # Convert logits to predicted class indices
    # Get the index of the max log-probability
    predicted_probs = torch.softmax(outputs, dim=-1) # (B, 4, 10)
    predicted_labels = torch.argmax(predicted_probs, dim=-1) # (B, 4)
    
    predicted_labels = predicted_labels.cpu() # Move predictions to CPU
    
    logger.info("📊 Visualizing predictions vs true labels for the batch...")
    
    num_images_in_batch = images.shape[0]
    num_cols = 4 # Display 4 grid images per row
    num_rows = (num_images_in_batch + num_cols - 1) // num_cols
    
    plt.figure(figsize=(num_cols * 3, num_rows * 3.5)) # Adjust size
    plt.suptitle("Phase 2 Predictions vs True Labels", fontsize=16, y=1.01)
    
    for i in range(num_images_in_batch):
        plt.subplot(num_rows, num_cols, i + 1)
        true_label_list = true_labels[i].tolist()
        pred_label_list = predicted_labels[i].tolist()
        
        title = f"True: {true_label_list}\nPred: {pred_label_list}"
        # Add checkmark or X for correctness
        is_correct = (true_labels[i] == predicted_labels[i]).all().item()
        title_prefix = "✅ " if is_correct else "❌ "
        
        imshow(images[i].cpu(), title=title_prefix + title)
        
    plt.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.show()
    
else:
    logger.error("❌ Cannot visualize results: Model or DataLoader not available.")

2025-04-28 17:28:31 | Backprop Bunch | INFO     | [1921474486.py:2] | Getting predictions for one batch...
2025-04-28 17:28:31 | Backprop Bunch | INFO     | [1921474486.py:18] | 📊 Visualizing predictions vs true labels for the batch...


/var/folders/s6/qfykwyn55ksgv8n9prqq55yc0000gq/T/ipykernel_48281/1921474486.py:39: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.98])
/var/folders/s6/qfykwyn55ksgv8n9prqq55yc0000gq/T/ipykernel_48281/1921474486.py:39: UserWarning: Glyph 10060 (\N{CROSS MARK}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.98])


## 5. Conclusion ✅

This notebook allowed us to load our trained Phase 2 ViT model and visualize its predictions on the 2x2 grid task. It helps confirm the model is learning correctly and identify potential areas for improvement or common failure modes before moving to the more complex Phase 3.